In [79]:
import torch
import pandas as pd
import json

scripts_dir = os.getcwd()
# Go up two level99ol,s
project_root = os.path.abspath(os.path.join(scripts_dir, '..', '..'))
sys.path.append(project_root)

import src.channel_model
importlib.reload(src.channel_model)
from src.channel_model import UIU


In [77]:
df = pd.read_csv("../data/synthesis/synthesized_data_UIU.csv")

In [78]:
df["U_R"] = df["U_R"].apply(lambda x: torch.tensor(json.loads(x)))

df["U_T"] = df["U_T"].apply(lambda x: torch.tensor(json.loads(x)))

df["H_bar"] = df["H_bar"].apply(lambda x: torch.tensor(json.loads(x)))

df["G"] = df["G"].apply(lambda x: torch.tensor(json.loads(x)))

df["V"] = df["V"].apply(lambda x: torch.tensor(json.loads(x)))

df["P"] = df["P"].apply(lambda x: torch.tensor(json.loads(x)))

In [80]:
class UIU_algorithm():

    def __init__(self, chan_gen, SNR, num_samples=10000):
        self.chan_gen = chan_gen
        self.n_R, self.n_T = chan_gen.generate().shape
        self.num_samples = num_samples
        self.H_bar = chan_gen.H_bar
        if torch.all(self.H_bar == 0):
            self.V = chan_gen.U_T
        else:
            U, S, VT = torch.linalg.svd(self.H_bar.T@self.H_bar)
            self.V = U
        self.P = torch.eye(self.n_T)
        self.SNR = SNR

    def H_gen(self):
        self.H = self.chan_gen.generate()
        return self.H

    def H_hat(self): return self.H @ self.V

    def h_hat(self, j): return self.H_hat()[:, j].reshape(-1, 1)

    def B(self, j):
        H_j_hat = torch.cat((self.H_hat()[:, :j], self.H_hat()[:, j+1:]), dim=1)
        P_new = torch.cat((self.P[:j, :], self.P[j+1:, :]), dim=0)  # Remove j-th row
        P_j = torch.cat((P_new[:, :j], P_new[:, j+1:]), dim=1)
        return torch.linalg.inv(torch.eye(self.n_R) + (self.SNR / self.n_T) * (H_j_hat @ P_j @ H_j_hat.T))
    
    def MMSE(self, j): return 1/(1 + self.P[j, j]*(self.SNR/self.n_T)*((self.h_hat(j).T)@self.B(j)@self.h_hat(j)))[0, 0]

    def MMSE_bar(self, j):
        s = 0
        for i in range(self.num_samples):
            self.H_gen()
            s += self.MMSE(j)
        return s/self.num_samples
    
    def Eq12(self, j):
        s = 0
        for i in range(self.num_samples):
            self.H_gen()
            s += (self.h_hat(j).T)@self.B(j)@self.h_hat(j)
        return (self.SNR*s)/(self.n_T*self.num_samples)

    def alg(self):
        flag2 = 0
        while True: 
            flag = 0
            #Allocation
            for k in range(8):
                diag = []
                den = (sum([(1 - self.MMSE_bar(i)) for i in range(self.n_T)])/self.n_T)
                for j in range(self.n_T):
                    diag.append((1 - self.MMSE_bar(j))/den)
                torch.diagonal(self.P).copy_(torch.tensor(diag))
            #Check
            rhs = sum([(1 - self.MMSE_bar(i)) for i in range(self.n_T)])/self.n_T
            for i in range(self.n_T):
                if abs(self.P[i, i]) <= 0.001 * self.n_T:
                    if self.Eq12(i) > rhs:
                        l = [self.Eq12(k) for k in range(self.n_T)]
                        idx = l.index(min(l))
                        self.P[idx, idx] = 0
                        flag = 1
                        flag2 = 1
                        break  
            if flag == 0: break
        return flag2, self.V, self.P

In [81]:
for i in range(len(df)):

    U_R = df["U_R"][i]
    U_T = df["U_T"][i]
    H_bar = df["H_bar"][i]
    G = df["G"][i]

    chan_gen = UIU(H_bar, U_R, U_T, G)
    algorithm = UIU_algorithm(chan_gen, SNR = 4, num_samples=10000)
    f, v, p = algorithm.alg()
    if f == 1:
        print("Entered second step")
        break
